In [76]:
import pandas as pd

Inflation csv from https://fred.stlouisfed.org/series/CPIAUCSL

In [77]:
inflation = pd.read_csv('../data/inflation_fred.csv')

In [78]:
inflation.head()

,observation_date,CPIAUCSL
0,1/1/1959,29.01
1,2/1/1959,29.00
2,3/1/1959,28.97
3,4/1/1959,28.98
4,5/1/1959,29.04


In [79]:
# changing to datetime format as the other dataset 
inflation['date'] = pd.to_datetime(inflation['observation_date'], format='%m/%d/%Y', errors='coerce')

In [80]:
# dropping unused columns and renaming 
inflation = inflation.drop(columns=['observation_date']).rename(columns={'CPIAUCSL': 'inflation'})

Unemployment csv from https://data.bls.gov/pdq/SurveyOutputServlet


In [82]:
unemployment = pd.read_csv('../data/unemployment_bls.csv', skiprows = 11)

In [83]:
unemployment.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1959,6.0,5.9,5.6,5.2,5.1,5.0,5.1,5.2,5.5,5.7,5.8,5.3
1,1960,5.2,4.8,5.4,5.2,5.1,5.4,5.5,5.6,5.5,6.1,6.1,6.6
2,1961,6.6,6.9,6.9,7.0,7.1,6.9,7.0,6.6,6.7,6.5,6.1,6.0
3,1962,5.8,5.5,5.6,5.6,5.5,5.5,5.4,5.7,5.6,5.4,5.7,5.5
4,1963,5.7,5.9,5.7,5.7,5.9,5.6,5.6,5.4,5.5,5.5,5.7,5.5


In [84]:
# month conversion abbv to month number to aid in datetime conversion
month_mapping = {'Jan': '01', 'Feb': '02', 'Mar': '03',
                 'Apr': '04', 'May': '05', 'Jun': '06',
                 'Jul': '07', 'Aug': '08', 'Sep': '09',
                 'Oct': '10', 'Nov': '11', 'Dec': '12'}

In [85]:
unemployment_long = pd.melt(unemployment, 
        id_vars = ['Year'],
        var_name = 'Month', 
        value_name= 'unemployment_rate'
        )

In [86]:
unemployment_long['month_num'] = unemployment_long['Month'].map(month_mapping)

In [87]:
unemployment_long['date'] =unemployment_long['Year'].astype(str) + '-' + unemployment_long['month_num'] 

In [88]:
unemployment_long['date'] = pd.to_datetime(unemployment_long['date'], format='%Y-%m', errors='coerce')

In [89]:
unemployment_long = unemployment_long.drop(columns = ['Year', 'Month', 'month_num'])

In [90]:
unemployment_long.head()

,unemployment_rate,date
0,6.0,1959-01-01
1,5.2,1960-01-01
2,6.6,1961-01-01
3,5.8,1962-01-01
4,5.7,1963-01-01


Monthly Consumer Debt from https://www.federalreserve.gov/datadownload/Download.aspx?rel=G19&series=be2df920f30707fd397c306408143a6c&lastObs=&from=&to=&filetype=csv&label=include&layout=seriescolumn&type=package

Revolving consumer debt is missing for the first 10 years, but was not as common during that time, as the first credit card was issued in the 1950s.

In [100]:
debt = pd.read_csv('../data/debt_fed.csv')

In [101]:
debt.head(15)

,Series Description,"Percent change of total consumer credit, seasonally adjusted at an annual rate","Percent change of total revolving consumer credit, seasonally adjusted at an annual rate","Percent change of total nonrevolving consumer credit, seasonally adjusted at an annual rate","Total consumer credit owned and securitized, seasonally adjusted level","Revolving consumer credit owned and securitized, seasonally adjusted level","Nonrevolving consumer credit owned and securitized, seasonally adjusted level","Total consumer credit owned and securitized, not seasonally adjusted level","Total securitized consumer credit, not seasonally adjusted level","Total consumer credit owned and securitized by depository institutions, not seasonally adjusted level",...,"Finance rate on personal loans at commercial banks, 24 month loan; not seasonally adjusted","Commercial bank interest rate on credit card plans, all accounts; not seasonally adjusted","Commercial bank interest rate on credit card plans, accounts assessed interest; not seasonally adjusted","New car average finance rate at auto finance companies, not seasonally adjusted (discontinued after January 2011)","Weighted-average maturity of new car loans at auto finance companies, not seasonally adjusted (discontinued after January 2011)","Loan-to-value ratio of new car loans at auto finance companies, not seasonally adjusted (discontinued after January 2011)","Average amount financed for new car loans at auto finance companies, not seasonally adjusted (discontinued after January 2011)","Average maturity of new car loans at finance companies, amount of finance weighted, not seasonally adjusted","Average finance rate of new car loans at finance companies, amount of finance weighted, not seasonally adjusted","Average amount financed for new car loans at finance companies, not seasonally adjusted"
0,Unit:,Percent,Percent,Percent,Currency,Currency,Currency,Currency,Currency,Currency,...,Percent,Percent,Percent,Percent,Months,Percent,Currency,Months,Percent,Currency
1,Multiplier:,1,1,1,1000000,1000000,1000000,1000000,1000000,1000000,...,1,1,1,1,1,1,1,1,1,1
2,Currency:,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
3,Unique Identifier:,G19/CCOUT/DTCTL_@%A_BA.M,G19/CCOUT/DTCTLR_@%A_BA.M,G19/CCOUT/DTCTLN_@%A_BA.M,G19/CCOUT/DTCTL.M,G19/CCOUT/DTCTLR.M,G19/CCOUT/DTCTLN.M,G19/CCOUT/DTCTL_N.M,G19/CCOUT/DTCNL_N.M,G19/CCOUT/DTCTLHD_N.M,...,G19/TERMS/RIFLPBCIPLM24_N.M,G19/TERMS/RIFSPBCICC_N.M,G19/TERMS/RIFSPBCICCI_N.M,G19/TERMS/H0.RIFLPCFAN_N.M,G19/TERMS/H0.DTCTLVNM_N.M,G19/TERMS/H0.DTCTLVNL_N.M,G19/TERMS/H0.DTCTLVNA_N.M,G19/TERMS/DTCTLVENM_N.M,G19/TERMS/RIELPCFAN_N.M,G19/TERMS/DTCTLVENA_N.M
4,Time Period,DTCTL_@%A_BA.M,DTCTLR_@%A_BA.M,DTCTLN_@%A_BA.M,DTCTL.M,DTCTLR.M,DTCTLN.M,DTCTL_N.M,DTCNL_N.M,DTCTLHD_N.M,...,RIFLPBCIPLM24_N.M,RIFSPBCICC_N.M,RIFSPBCICCI_N.M,H0.RIFLPCFAN_N.M,H0.DTCTLVNM_N.M,H0.DTCTLVNL_N.M,H0.DTCTLVNA_N.M,DTCTLVENM_N.M,RIELPCFAN_N.M,DTCTLVENA_N.M
5,1943-01,NaN,NaN,NaN,6577.83,NaN,6577.83,6550,NaN,2554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1943-02,-20.94,NaN,-20.94,6463.04,NaN,6463.04,6276,NaN,2444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1943-03,-42.49,NaN,-42.49,6234.21,NaN,6234.21,6099,NaN,2376,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1943-04,-20.88,NaN,-20.88,6125.75,NaN,6125.75,6065,NaN,2324,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1943-05,-37.12,NaN,-37.12,5936.26,NaN,5936.26,5967,NaN,2278,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
# had to strip columns to be able to find them
debt.columns = debt.columns.str.strip()

debt = debt[['Series Description', 
             'Revolving consumer credit owned and securitized, seasonally adjusted level', 
             'Nonrevolving consumer credit owned and securitized, seasonally adjusted level', 
             'Total consumer credit owned and securitized, seasonally adjusted level']]

In [103]:
# removing the first five rows to remove informational rows
debt = debt.iloc[5:]

In [104]:
debt['date'] = pd.to_datetime(debt['Series Description'], format='%Y-%m', errors='coerce')

In [105]:
debt = debt.drop(columns=['Series Description']).rename(columns = {
    'Revolving consumer credit owned and securitized, seasonally adjusted level': 'revolving_credit',
    'Nonrevolving consumer credit owned and securitized, seasonally adjusted level': 'nonrevolving_credit',
    'Total consumer credit owned and securitized, seasonally adjusted level': 'total_credit'
})

In [106]:
debt.head()

,revolving_credit,nonrevolving_credit,total_credit,date
5,NaN,6577.83,6577.83,1943-01-01
6,NaN,6463.04,6463.04,1943-02-01
7,NaN,6234.21,6234.21,1943-03-01
8,NaN,6125.75,6125.75,1943-04-01
9,NaN,5936.26,5936.26,1943-05-01


Personal Consumption Expenditures from https://apps.bea.gov/iTable/

In [108]:
# years and months are on different rows, so did not bring in a header to be able to combine the years and month
per_cons = pd.read_csv('../data/pce_bea.csv', skiprows = 3, header = None)

In [109]:
# separating the years and months and then combining them
years = per_cons.iloc[0]
months = per_cons.iloc[1]
combined_date = months + ' ' + years

In [110]:
date = pd.to_datetime(combined_date, format = '%b %Y', errors='coerce')

In [ ]:
# assign date as the header 
per_cons.columns = date

In [113]:
# changing to a string to be able to rename column 
per_cons.columns = per_cons.columns.astype(str)
per_cons.columns.values[1] = 'description'

In [114]:
# stripping whitespace from the description column to be able to find row needed
per_cons['description'] = per_cons['description'].str.strip()

In [115]:
per_cons = per_cons[per_cons['description'] == 'Personal consumption expenditures']

In [118]:
per_cons.head(35)

,NaT,description,1959-01-01,1959-02-01,1959-03-01,1959-04-01,1959-05-01,1959-06-01,1959-07-01,1959-08-01,...,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01,2025-04-01
30,29,Personal consumption expenditures,306.1,309.6,312.7,312.2,316.1,318.2,317.8,320.2,...,19866.3,19905.0,20044.1,20123.2,20235.1,20408.1,20389.0,20469.3,20621.8,20669.5


In [117]:
per_con_long = pd.melt(
    per_cons, 
    id_vars = None,
    var_name = 'date',
    value_name = 'pce'
)

In [119]:
per_con_long.head()

,date,pce
0,NaT,29
1,description,Personal consumption expenditures
2,1959-01-01,306.1
3,1959-02-01,309.6
4,1959-03-01,312.7


In [120]:
# removing rows not needed
per_con_long = per_con_long.iloc[2:]

In [121]:
per_con_long['date'] = pd.to_datetime(per_con_long['date'])

In [122]:
# merging all datasets together
combined_df = per_con_long.merge(
    inflation, 
    how='left', 
    on='date'
).merge(
    unemployment_long, 
    how='left', 
    on='date'
).merge(
    debt, 
    how='left', 
    on='date'
)

In [127]:
combined_df.head()

,date,pce,inflation,unemployment_rate,revolving_credit,nonrevolving_credit,total_credit
0,1959-01-01,306.1,29.01,6.0,NaN,48961.16,48961.16
1,1959-02-01,309.6,29.00,5.9,NaN,49513.71,49513.71
2,1959-03-01,312.7,28.97,5.6,NaN,50007.73,50007.73
3,1959-04-01,312.2,28.98,5.2,NaN,50463.43,50463.43
4,1959-05-01,316.1,29.04,5.1,NaN,51007.24,51007.24


In [128]:
# saving csv to use in other notebook
combined_df.to_csv('../data/dataformodel.csv', index=False)